<a href="https://colab.research.google.com/github/victorprieto2265/cornerstone/blob/main/Cornerstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO + RFE



TODO

* Fix import problem again... I'm uploading the playoffs data file but it keeps seeing only the prelims one...
 * update: the cause of this bug was discovered - cornerstone will only use the first uploaded file even if additional spreadsheets with the same name are uploaded, it ignores the copy. The manual workaround is to delete the first original copy if changes are made.
* Import computer modern typeface for use (consider open sans or interstate as an alternative).
* Detect and split up tables that run over multiple pages (likely documents other than individual schedules for any 32+ team tournaments).
* Qr_toggle keeps returning as false despite being updated differently in prelims_input, can't figure out the github new file issue!
* Write new error check for duplicates and length limits. The old one is currently commented out.
* There's an encoding problem for text: FPDF2 only accepts Latin-1, and I don't know enough around encoding except to tell users not to use anything outside Latin-1. Notably, an em dash and an ellipsis character will cause an encoding problem (the latter is what led me to identify this problem). This could affect all text inputs anywhere, not just individual text schedule inputs.
* Finish the crossover templates in rr_templates.xlsx at some point. Right now crossovers are kind of a mess.

RFE

* Someone has pointed out 3-letter team codes are unnecessary for everything outside of the complete schedule. It's included in the team index, but not actually used in any documents other than the complete schedule. After Cornerstone has been in use for some time, re-evaluate the necessity of the 3-letter code, and correspondingly the complete schedule.
 * Cornerstone relies on 3-letter codes for a lot of functions. To avoid rewriting half the program, instead assign a unique short string to each team upon input - for instance, T1 through Tn would be fine.
* In class definition for individual room schedules, add a row for score confirmation + signature.



# Import modules / libraries

In [1]:
!pip install fpdf2
!pip install qrcode
!pip install texttable

import sys
import pandas as pd
import warnings
warnings.simplefilter('default', DeprecationWarning)
from fpdf import FPDF
import fpdf
from base64 import b64encode
import IPython
from IPython.display import display, HTML
import qrcode
import base64
import requests
from google.colab import files
import texttable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Cornerstone Input

This section imports all of the input files, except tournament format,
which is handled by the tournament_format script.

TODO uncomment sys.exit at end of this script for testing

TODO figure out how to handle blank spaces and/or none in input file...
There was an unusual error where integers were being converted to floats
if one of the cells in a range was empty instead of string 'none'.

TODO apparently bracket names (and presumably other things) are case-sensitive.

TODO currently a global variable for checking for errors, which is a no no?


In [129]:
error = 0

# checks for duplicates and (optional) character limits
def error_check(alist, listname, max_length=float('inf'), max_duplicates=1):
    global error  # TODO this is apparently a no no, but works for now
    print('\n')
    # duplicate checker
    seen = {}
    duplicates = []
    for item in alist:
        if item not in seen:
            seen[item] = 1
        else:
            seen[item] += 1
            if seen[item] > max_duplicates:
                duplicates.append(' - ' + str(item))
    if not duplicates:
        if max_duplicates > 1:
            print(f'No excess duplicates detected for {listname}.')
        else:
            print(f'No duplicates detected for {listname}.')
    else:
        # error = True
        error += 1
        print(f'\nDuplicates detected for {listname}:')
        print(*duplicates, sep='\n')

    # character length checker
    long_items = [' - ' + item for item in alist if len(item) > max_length]
    if not long_items:
        print(f'No character lengths exceeded for {listname}.')
    else:
        error += 1
        # error = True
        print(f'Character lengths exceeded for {listname}:')
        print(*long_items, sep='\n')

Function definition: analyze sheet and generate list from pandas dataframe.

In [130]:
def analyze_input(sheet_name, df_dict):
    df = df_dict[sheet_name]
    temp_list = df.values.tolist()  # note: temp_list may be multiple lists!
    return temp_list

---

Upload the prelims_data excel spreadsheet from local computer, and create dictionary containing the sheet name and each dataframe.

TODO For now, this is commented out. While testing Cornerstone as I'm porting over to Colab, I will be directly pulling the excel file from a dedicated location in github.

In [131]:
file_upload = files.upload()
file_upload = list(file_upload.keys())[0]

sheet_names = ['tournament format', 'list of teams', 'team codes',
                'group names', 'room assignments',
                'QR codes', 'text input']
df_dict = pd.read_excel(file_upload,
                               sheet_name=sheet_names)

Saving playoffs_data_columbia.xlsx to playoffs_data_columbia.xlsx


In [132]:
# sheet_names = ['tournament format', 'list of teams', 'team codes',
#                 'group names', 'room assignments',
#                 'QR codes', 'text input']
# url = "https://github.com/victorprieto2265/cornerstone/blob/main/data%20input/prelims_data.xlsx?raw=true"
# df_dict = pd.read_excel(url,
#                                sheet_name=sheet_names)


Determine phase of the tournament based on import filename.

In [133]:
# check to make sure filename has the correct prefix for tournament phase
acceptable_file_prefixes = ('prelims', 'playoffs', 'super')
if not file_upload.startswith(acceptable_file_prefixes):
  print('\nWARNING: filename must start with one of the following:')
  for prefix in acceptable_file_prefixes:
    print(prefix)
  sys.exit()

if file_upload.startswith('prelims'):
  tournament_phase = 'prelims'
elif file_upload.startswith('playoffs'):
  tournament_phase = 'playoffs'
elif file_upload.startswith('super'):
  tournament_phase = 'superplayoffs'

Using the analyze_input function, create a list from each tab of the spreadsheet corresponding to the relevant information (e.g. a list of room assignments). If in playoffs or superplayoffs, also create the relevant lists there.

In [134]:
list_of_teams = analyze_input('list of teams', df_dict)
team_codes = analyze_input('team codes', df_dict)
room_assignments = analyze_input('room assignments', df_dict)
qr_codelist = analyze_input('QR codes', df_dict)
textlist = analyze_input('text input', df_dict)
tournament_format = analyze_input('tournament format', df_dict)

group_names = analyze_input('group names', df_dict)
prelim_group_names = [row[0] for row in group_names]

if tournament_phase in ['playoffs', 'superplayoffs']:
  playoff_bracket_names = [row[1] for row in group_names]
  playoff_bracket_names = [x for x in playoff_bracket_names
                            if str(x) != 'nan']
  playoff_list_of_teams = list_of_teams

if tournament_phase == 'superplayoffs':
  super_bracket_names = [row[2] for row in group_names]
  super_bracket_names = [x for x in super_bracket_names
                            if str(x) != 'nan']
  super_list_of_teams = list_of_teams


Sometimes, tournaments don't have enough teams to warrant more than one prelim group. We'll do a quick check to see if that's the case.

In [135]:
if len(prelim_group_names) == 1:
  single_prelim_group = True
else:
  single_prelim_group = False

A dictionary containing general information about the tournament is also generated from the tournament_format sheet.

It's necessary to convert the datetime value from excel to a more suitable string before printing.

In [136]:
format_dict = {}
for row in tournament_format:
    key = row[0]  # item, e.g. "tournament name"
    value = row[1]  # input, e.g. "2022 NSC"
    format_dict[key] = value

# convert Excel datetime format to more suitable date string
date = format_dict['tournament date'].strftime('%B %d, %Y')
format_dict['tournament date'] = date

Next is the analysis section for the text and QR code inputs. Did the user turn these on? If not, these sections are skipped.

In [137]:
if format_dict['QR codes'] == 'Y':
    qr_toggle = True

    qr_names = [code[0] for code in qr_codelist]
    qr_codes = [code[2] for code in qr_codelist]
    qr_captions = [code[3] for code in qr_codelist]

else:
    qr_toggle = False
    qr_codes = False
    qr_captions = False

if format_dict['text'] == 'Y':
    text_toggle = True
    texts = [text[1] for text in textlist]

else:
    text_toggle = False
    texts = False

More dictionary creation here: this time, for team names and team codes.

RFE get rid of team codes if the user is okay with foregoing the complete team schedule in favor of individual team schedules.

In [138]:
# key is team name
# value is team code
# also created a dictionary where the key/value pairs are swapped
team_code_dict = {}
code_team_dict = {}
for row in team_codes:
    team_name = row[0]
    team_code = row[1]
    team_code_dict[team_name] = team_code
    code_team_dict[team_code] = team_name

# key is prelim group and a number (i.e. Accra1)
# value is corresponding room (i.e. Grand Ballroom A is Accra1)
# also created dictionary where key is a playoff bracket instead
prelim_room_dict = {}
playoff_room_dict = {}
super_room_dict = {}
for i in room_assignments:
    key = i[1] + str(i[2])
    value = i[0]
    if tournament_phase == 'prelims':
        prelim_room_dict[key] = value
    elif tournament_phase == 'playoffs':
        playoff_room_dict[key] = value
    elif tournament_phase == 'superplayoffs':
        super_room_dict[key] = value

Last is the error check section, to see if any of the data submitted by the user was improperly formatted.

TODO this whole section needs to be updated... And perhaps error checks should be distributed elsewhere in the program. It's not working well.

In [139]:
# TODO modify max_duplicates to be flexible on schedule and not hard coded
# or perhaps get rid of max duplicates since it doesn't make a lot of sense

# error_check([sublist[0] for sublist in list_of_teams],
#             'the team names in list_of_teams',
#             max_length=26)

# error_check([sublist[2] for sublist in list_of_teams],
#             'the prelim groups in list_of_teams',
#             max_length=15,
#             )

# error_check([sublist[1] for sublist in team_codes],
#             'the team codes in team_codes',
#             max_length=4)

# try:
#     error_check(prelim_group_names,
#                 'the groups in prelim_group_names',
#                 max_length=15)
# except:  # TODO bare except
#     None

# try:
#     error_check(playoff_bracket_names,
#                 'the brackets in playoff_bracket_names',
#                 max_length=15)
# except:  # TODO bare except
#     None

# error_check([sublist[0] for sublist in room_assignments],
#             'the list of rooms in room_assignments',
#             max_length=15)

if error == 0:
    print('\n*** No errors detected upon import. ***')
else:
    print('\n*** Errors detected during import! ***')
    sys.exit()


*** No errors detected upon import. ***


# Function Definitions

Function for removing duplicates from a list.

In [140]:
def remove_duplicates(alist):
    newlist = []
    [newlist.append(x) for x in alist if x not in newlist]
    return newlist

Functions for reading playoff codes 1 and 2 and returning the correctly sorted list of teams.

In [141]:
def code_1_scheduler(teamlist, code):
  code_1 = code.split(',')
  newlist = []
  for i in code_1:
    newlist.append(teamlist[int(i)-1])
  return newlist


def code_2_scheduler(teamlist, code):
  code_2 = code.split(',')
  code_2 = [int(i) for i in code_2]
  newlist = []
  for index, element in enumerate(code_2):
    slice1 = sum(code_2[0:index])
    slice2 = slice1 + element
    newlist.append(teamlist[slice1:slice2])
  return newlist

Visual debugging this program often requires visualization of tables. This function is a quick way of displaying pretty tables.

In [142]:
def visual_debugger(table):
  # Create texttable object
  tableObj = texttable.Texttable(max_width=250)

  # Set columns
  tableObj.set_cols_align(["l", "r", "c"])

  # Set datatype of each column
  tableObj.set_cols_dtype(["a", "i", "t"])

  # Adjust columns
  tableObj.set_cols_valign(["t", "m", "b"])
  # col_widths = [5 for col in table[0]]
  # print(col_widths)
  # tableObj.set_cols_width(col_widths)

  # Insert rows
  tableObj.add_rows(table)

  # Display table
  print(tableObj.draw())


original function_definitions.py

In [143]:
# import time
# import sys
# from pathlib import Path
# from pylatex import (Document, Tabularx, Command,
#                       NewPage, HugeText, Package,
#                      VerticalSpace)
# from pylatex.utils import NoEscape
# from cornerstone_input import format_dict

# # place additional modules here

# header = """

# Depository for a bunch of class and function definitions shared across
# prelim/playoff scripts.

# TODO what if snake seed is fed a list with odd number of teams?

# TODO what if qr_count ≠ 3?

# Created on Mon Nov 1 13:53:44 2021 Eastern Time

# @author: Victor Prieto

# """


# # starts program runtime
# start_time = time.time()
# print('\n', header)
# print('start time: %s' % time.ctime())

# # %% function definitions





# def split_list(alist, wanted_parts=1):
#     length = len(alist)
#     return [alist[i*length // wanted_parts: (i+1)*length // wanted_parts]
#             for i in range(wanted_parts)]


# def snake_seed(list_of_teams, bracket_count):
#     """
#     Parameters
#     ----------
#     list_of_teams : list
#         A list of teams for reordering into playoff brackets by snake seeding.
#     bracket_count : int
#         Number of playoff brackets to snake seed across.

#     Returns
#     -------
#     newlist : list
#         Same list of teams accepted as input, but reordered by snake seeding.
#         The list is also split into the number of playoff brackets.
#         Example:
#             (1, 2, 3, 4... 23, 24)
#             snake seeded across four brackets becomes
#             ((1, 8, 9, 16, 17, 24), (2, 7, 10, 15, 18... 12, 13, 20, 21))

#     """

#     # generates sequence of indices for snake seeding
#     sequence_length = 2 * bracket_count
#     reps = len(list_of_teams) / sequence_length
#     sequence = []
#     for i in range(0, int(sequence_length/2)):
#         for j in range(0, int(reps)):
#             sequence.append(i+j*sequence_length)
#             sequence.append((sequence_length-1-i)+j*sequence_length)

#     # quits program if odd number of teams in list_of_teams
#     if (len(list_of_teams)) % 2 == 1:
#         print('ODD NUMBER OF TEAMS DETECTED')
#         sys.exit()

#     # use sequence of indices to generate new list of teams
#     newlist = (list_of_teams[index] for index in sequence)
#     return newlist


# def alternating_rows(doc, color):
#     return doc.append(NoEscape(r'\rowcolors{3}{white}{%s}' % color))


# def duplicate_checker(alist):
#     seen = set()
#     for x in alist:
#         if x in seen:
#             return True
#         seen.add(x)
#     return False


# # %% classes, mostly not in use
# # TODO figure out class inheritance for team/prelim_team/playoff_team
# class team:
#     def __init__(self,
#                  name,
#                  code):
#         self.name = name
#         self.code = code


# class prelim_team(team):
#     def __init__(self,
#                  name,
#                  code,
#                  prelim_group,
#                  prelim_seed):
#         super().__init__(name, code)
#         self.prelim_group = prelim_group
#         self.prelim_seed = prelim_seed


# class playoff_team():
#     def __init__(self,
#                  name,
#                  code,
#                  prelim_group,
#                  playoff_bracket,
#                  playoff_seed):  # playoff seed also the prelim finish
#         self.name = name
#         self.code = code
#         self.prelim_group = prelim_group
#         self.playoff_bracket = playoff_bracket
#         self.playoff_seed = playoff_seed



# # %% prints runtime
# print('end time: %s' % time.ctime())
# print("--- %s seconds ---" % '%.3f' % (time.time() - start_time))
# print("--- %s minutes ---" % '%.3f' % (time.time()/60 - start_time/60))

# Standard Scheduler Function

Created on Mon Nov 22 2021

Definition of functions used for creating a table for a round robin schedule,
between 4 and 14 teams.

TODO if possible, fix pylatex table lines and shading in this function

First: retrieve the round robin templates stored on the cornerstone github page. The templates will automatically populate a dictionary.

Each key is the name of the sheet corresponding to the round robin template: rr_6 is a basic 6 team round robin, rr_12 is for 12 teams, rr_crossover_A8 is the 8 team playoff roundrobin including crossovers (there are two possible crossovers, A and B, based on [Dwight Wynne's spreadsheet of schedule formats](https://https://docs.google.com/spreadsheets/d/1BPF6jFvhkh5qe7eesGFGU9KH6FVps2tS3YeseFocpIE/edit#gid=16)).

The dictionary values are the pandas dataframes for each template.

These templates will be used in the standard scheduler function and retrieved based on the key name.

In [144]:
url = "https://github.com/victorprieto2265/cornerstone/blob/main/templates/rr_schedules/rr_templates.xlsx?raw=true"
rr_template_dict = pd.read_excel(url,
                                 sheet_name = None)

Definition of the standard scheduler function is below.

In [145]:
def standard_schedule(group_name, teamcode_dict, room_dict, roundstart=1,
                      multiple_rr=1, crossover=False, superplayoff=False):
    """


    Parameters
    ----------
    groupname : string
        String of the prelim group or playoff bracket name.
        Example: Lilongwe
    teamcode_dict : dict
        k : v = a group/bracket code : teamcode corresponding to that code
        example: Lilongwe4 = MBA
    room_dict : dict
        k : v = a group/bracket code : room corresponding to that code
        example: Lilongwe2 = Suite 1132
    roundstart : int, optional
        Number for the first round in the schedule. The default is 1.

    Returns
    -------
    schedule_grid : list
        Latex-friendly table. First row = table header, remaining rows =
        schedule for each round.

    """

    # produce teamlist
    teamlist = [k for k, v in list(teamcode_dict.items())
                if k.startswith(group_name)]
    sorted_list = sorted(teamlist)
    teamlist = [teamcode_dict[i] for i in sorted_list]

    # identify number of teams in teamlist and appropriate number of rooms
    teamcount = len(teamlist)
    roomcount = teamcount // 2

    if teamcount <= 2:
      print('Unable to produce round robins smaller than 3.')
      sys.exit()

    if teamcount >= 15:
      print('Unable to produce round robins larger than 14.')
      sys.exit()

    # retrieve correct rr_template from rr_template_dict
    if multiple_rr >= 2 or not isinstance(multiple_rr, int):
      key = f'rr_{teamcount}_{multiple_rr}x'
    elif crossover is False:
      key = f'rr_{teamcount}'
    else:
      key = f'rr_crossover_{crossover}{teamcount}'

    try:
      rr_schedule = rr_template_dict[key].values.tolist()
    except KeyError:
      print('Warning! Round robin template not found. Please check your '
            + 'input and try again.')
      sys.exit()
    

    # produce roomlist
    roomlist = [k for k, v in list(room_dict.items())
                if k.startswith(group_name)]
    roomlist = [room_dict[i] for i in sorted(roomlist)]

    # populate header with room names
    rooms = []
    header = ['']
    for index in range(roomcount):
        room = roomlist[index]
        header.append(room)

    # the last element in the header should be BYE if teamcount is odd
    if teamcount % 2 == 1:
        header.append('BYE')

    # use rr_schedule template and roundstart to produce rest of schedule grid
    schedule_grid = [header]
    for round_num, row in enumerate(rr_schedule):
        round_num += roundstart
        new_row = [f'Round {round_num}']
        for index in row:
            code = group_name + str(index)
            team = teamcode_dict[code]
            new_row.append(team)
        schedule_grid.append(new_row)

    return schedule_grid

# Individual Schedule Functions 

This section contains the functions that extract individual team and room schedules from a schedule grid.

Use get_team_list() and get_room_list() to generate a list of teams and a list of rooms from a schedule grid.

In [146]:
def get_team_list(schedule_grid):
    team_list = sorted(schedule_grid[1][1:])
    return team_list


def get_room_list(schedule_grid):
    # remove first element in header, which should be an empty string
    header = schedule_grid[0][1:]
    
    # retrieve room list
    room_list = [i for i in header]
    if 'BYE' in header:
        room_list[-1] = 'BYE'
    return room_list

For a given team in a given round, what team do they play and where? Use find_opponent().


In [147]:
def find_opponent(team, row, room_list, code_dict):
    index = row.index(team)
    room = room_list[index]
    if row[-1] == team and room_list[-1] == 'BYE':
        opponent = 'BYE'
    elif index % 2 == 0:
        opponent_code = row[index+1]
        opponent = code_dict[opponent_code]
    else:
        opponent_code = row[index-1]
        opponent = code_dict[opponent_code]
    return room, opponent

what teams are in a room at a given round?

In [148]:
def find_teams(row, room, roomlist, code_dict):
    index = roomlist.index(room)
    teams = row[2*index:2*index+2]
    teams = [*map(code_dict.get, teams)]
    return sorted(teams)  # alphabetically sorts teams

The clean_up_grid function is sometimes used to remove the header row (room list) and the round number column on the left.

In [149]:
def clean_up_grid(schedule_grid):
    no_header = schedule_grid[1:]
    basic_schedule_grid = []
    for index, row in enumerate(no_header):
        basic_schedule_grid.append(row[1:])

    return basic_schedule_grid

The specific_team_scheduler function is the key for generating a schedule grid for a single team.

In [150]:
def specific_team_scheduler(team, schedule_grid, room_list, code_dict,
                            round_start=1):
    
    # generate room_list with duplicated values
    # need this for finding an opponent at a particular index
    dup_room_list = [room for room in room_list for _ in (0, 1)]

    # how many rounds in the grid?
    rounds = len(schedule_grid)

    specific_grid = [['', 'Room Name', 'Opponent']]
    for i in range(0, rounds):
        room, opponent = find_opponent(team,
                                       schedule_grid[i],
                                       dup_room_list,
                                       code_dict)

        specific_grid.append([f'Round {round_start+i}', room, opponent])

    return specific_grid

Like specific_team_scheduler(), specific_room_scheduler is the key for generating the schedule for an individual room.

In [151]:
def specific_room_scheduler(room, schedule_grid, room_list,
                            code_dict, round_start=1):

    specific_grid = [['', 'Team 1', 'Team 2']]
    for index, row in enumerate(basic_schedule_grid):
        teams = find_teams(row, room, room_list, code_dict)
        if room == 'BYE':
            return 'BYE'
        specific_grid.append([f'Round {round_start+index}',
                              teams[0], teams[1]])

    return specific_grid

# Class Definitions for fpdf2

There are five types of tables that Cornerstone will need to generate:
*  the full schedule
*  the individualized team schedule
*  the individualized room schedule
*  the team index (includes toggle for prelims/playoffs/superplayoffs)
*  the playoff quickstart schedule (first playoff room for all teams)

The last one will not be needed for the prelim writer, but will be necessary for playoffs (and superplayoffs if needed).

For all of these tables, we will create a unique class via fpdf2 for writing the table onto a PDF. These five classes are at the bottom of this section. First, we will define a few templates used across the printed documents:

*  the title page
*  the header
*  the footer


First is the title page to start each document. A nice touch, albeit not 100% necessary and may add a toggle for this later.

Important: this is the first instance of the PDF class. Therefore, the argument is FPDF, while all other later class instances have PDF instead to build on the existing class.

In [152]:
class PDF(FPDF):

    def title_page(self, tournament_name, tournament_location, tournament_date,
                   document_title):
                
        # Setting font: bold times 11, black
        self.set_text_color(0)

        # move cursor 6.2 cm down the page
        self.set_y(62)

        # write the tournament name, location, and date
        self.set_font("times", size=16)
        self.cell(0, None, tournament_name, align="C",
                  new_x="LEFT", new_y="NEXT")
        self.ln(10)  # line break

        self.set_font("times", size=12)
        self.cell(0, None, tournament_location, align="C")
        self.ln(10)  # line break
        self.cell(0, None, tournament_date, align="C")
        self.ln(45)  # line break

        self.set_font("times", size=24)
        self.cell(0, None, document_title, align="C")

In [175]:
class PDF(PDF):

    def page_header(self, tournament_name, tournament_date):
                
        # Setting font: bold times 11
        font_size = 11
        self.set_font("times", style="I", size=font_size)

        # Print tournament name in upper left:
        self.cell(None, 2, tournament_name, new_x="LEFT")

        # Create empty cell with bottom border (can modify for centered text)
        self.cell(0, None, " ", "B", align="C")

        # Print tournament date in upper right
        self.cell(0, 2, tournament_date, align="R")

        # Performing a line break:
        self.ln(20)

    def page_footer(self):
        
        # # putting in a logo:
        # self.image("../docs/fpdf2-logo.png", 10, 8, 33)
        
        # Position cursor at left and 26.04 cm from bottom:
        # self.set_y(0, 260.4)
        self.set_y(-27)

        # Setting font: times italic 11
        font_size = 11
        self.set_text_color(0)
        self.set_font("times", style="I", size=font_size)
        
        # Print footer text:
        self.cell(0, 6, "Created by Cornerstone.", border="B", align="R")


## Full Schedule

This class generates the tables used for the full schedule.

In [154]:
class PDF(PDF):

    def full_schedule(self, header, rows,
                      header_color=(135, 135, 135), row_color=(210, 210, 210)):

      header_count = len(header)
      if header[-1] == 'BYE':
        cell_count = header_count * 2 - 2
      else:
        cell_count = header_count * 2 - 1
      col_width = pdf.epw / cell_count  # distribute columns evenly

      header_widths = [col_width * 2] * (header_count - 1)  # populate list with col widths
      header_widths.insert(0, col_width)
      if header[-1] == 'BYE':
        header_widths[-1] = col_width

      cell_widths = [col_width] * cell_count

      self.set_fill_color(*header_color)  # * required to unpack tuple
      self.set_text_color(255)
      self.set_draw_color(36, 36, 36)
      self.set_line_width(0.3)
      self.set_font(style="B", size=14)
      for col_width, heading in zip(header_widths, header):
          self.cell(col_width, 8, heading, border=1, align="C", fill=True)
      self.ln()

      # Color and font restoration:
      self.set_fill_color(*row_color)  # * required to unpack tuple
      self.set_text_color(0)
      self.set_font_size(12)
      fill = False
      for row in rows:
        for index, item in enumerate(row):          
          if index == 0:
            self.set_font(style="B")
          else:
            self.set_font()
          self.cell(cell_widths[index],
                    7,
                    item,
                    border="LR",
                    align="C",
                    fill=fill)
        self.ln()
        fill = not fill

      self.set_fill_color(*header_color)  # * required to unpack tuple
      self.set_text_color(255)
      self.set_draw_color(36, 36, 36)
      self.set_line_width(0.3)
      self.set_font(style="B", size=14)
      for col_width, heading in zip(header_widths, header):
          self.cell(col_width, 8, heading, border=1, align="C", fill=True)

      self.ln(14)

## Individual Team/Room Schedule

This class is used to generate the tables for individual team and room schedules, which are nearly identical except the room schedules will have a confirmation box for team scores and signatures.

In [155]:
class PDF(PDF):

    def individual_schedule(self, header, rows, signature=False,
                      header_color=(135, 135, 135), row_color=(210, 210, 210)):

      # start table at 15.45, draw cells 33 + 66 + 66 for each column
      # right margin will also be 15.45

      self.set_x(25.45)
      cell_widths = [33, 66, 66]

      self.set_fill_color(*header_color)  # * required to unpack tuple
      self.set_text_color(255)
      self.set_draw_color(36, 36, 36)
      self.set_line_width(0.3)
      self.set_font(style="B", size=14)
      for cell_width, heading in zip(cell_widths, header):
          self.cell(cell_width, 8, heading, border=1, align="C", fill=True)
      self.ln()
      self.set_x(25.45)

      # Color and font restoration:
      self.set_fill_color(*row_color)  # * required to unpack tuple
      self.set_text_color(0)
      self.set_font()
      fill = False
      if len(rows) >= 12:
        font_size = 10
        row_height = 6
      else:
        font_size = 12
        row_height = 7
      for row in rows:
        for index, item in enumerate(row):          
          if index == 0:
            self.set_font(style="B", size=font_size)
          else:
            self.set_font(size=font_size)
          self.cell(cell_widths[index],
                    row_height,
                    item,
                    border="LR",
                    align="C",
                    fill=fill)
        fill = not fill  # I am pretty sure this is what alternates row colors!

        self.ln()
        self.set_x(25.45)

        # RFE add row for team score + signature confirmation
        if signature == True:
          pass

      self.set_fill_color(*header_color)  # * required to unpack tuple
      self.set_text_color(255)
      self.set_draw_color(36, 36, 36)
      self.set_line_width(0.3)
      self.set_font(style="B", size=14)
      for cell_width, heading in zip(cell_widths, header):
          self.cell(cell_width, 8, heading, border=1, align="C", fill=True)

      self.ln(14)

## Team Index

This class is for making the tables for the team index documents.

# Prelim Scheduler

This section reads the list of prelim teams and information from the tournament format dictionary to produce a set of prelim schedules.

**This is the start of the prelim scheduling section.  Every block starts with %%skip $should_skip and should_skip is set to True if we are running a playoff or superplayoffs input file.**

In [156]:
file_upload = files.upload()

%load_ext skip_kernel_extension

if tournament_phase != 'prelims':
  should_skip = True
else:
  should_skip = False

Saving skip_kernel_extension.py to skip_kernel_extension (7).py
The skip_kernel_extension extension is already loaded. To reload it, use:
  %reload_ext skip_kernel_extension


A series of lists and dictionaries are created here, which will be useful at various points throughout the program:
* prelim_group_names: a list of prelim group names
* prelim_team_dict: a dictionary where every prelim group and seed is matched up with a group name + seed
* prelim_teamcode_dict: identical, but a teamcode instead of a full team name
* team_group_dict: same as prelim_team_dict, but the key/value pairs are swapped
* teamcode_group_dict: again, identical but with a teamcode instead of full team name
* prelim_room_dict: matches a prelim group and room assignment (e.g. group 2, room 3) with an actual room name

In [157]:
%%skip $should_skip

prelim_group_names = [sublist[1] for sublist in list_of_teams]
prelim_group_names = remove_duplicates(prelim_group_names)

# key is a prelim group name and a seed (i.e. Belmopan6)
# value is the corresponding team (i.e. Great Valley A for Belmopan6)
# also created a dictionary where values are team codes (i.e. GVA)
prelim_team_dict = {}
prelim_teamcode_dict = {}
team_group_dict = {}
teamcode_group_dict = {}
for i in list_of_teams:
    # key = i[2] + str(i[3])
    key = i[1] + str(i[2])
    value = i[0]  # team name
    prelim_team_dict[key] = value
    # also created dictionary where k/v pairs are swapped
    team_group_dict[value] = key
    # value = i[1]  # team code
    value = team_code_dict[value]
    prelim_teamcode_dict[key] = value
    # also created dictionary where k/v pairs are swapped
    teamcode_group_dict[value] = key

# key is prelim group and a number (i.e. Accra1)
# value is corresponding room (i.e. Grand Ballroom A is Accra1)
# also created dictionary where key is a playoff bracket instead
prelim_room_dict = {}
for i in room_assignments:
    key = i[1] + str(i[2])
    value = i[0]
    prelim_room_dict[key] = value

In [158]:
%%skip $should_skip

full_schedule_grid = []

# playoffs input doesn't have multiple_rr or is set to non-integer...
# set multiple_rr = 1
try:
  multiple_rr = format_dict['repeat round robin']
except KeyError:
  multiple_rr=1
if not isinstance(multiple_rr, int):
  multiple_rr = 1

# perform this process for each prelim group, add to full_schedule_grid
for group_name in prelim_group_names:
  schedule_grid = (standard_schedule(group_name,
                                     prelim_teamcode_dict,
                                     prelim_room_dict,
                                     multiple_rr=multiple_rr))
  full_schedule_grid.append(schedule_grid)

# # for visual debugging
# for index, prelim_group in enumerate(prelim_group_names):
#     print(f'\nPrelim Group: {prelim_group}')
#     print(*full_schedule_grid[index], sep='\n')

# Playoff Scheduler

This section ranks teams by prelim group finish, then PPB. The list is then provided to the playoff scheduler.

**This is the start of the playoff scheduling section. Every block will run %%skip $should_skip if the tournament phase is set to prelims or playoffs.**

In [159]:
if tournament_phase != 'playoffs':
  should_skip = True
else:
  should_skip = False

Once again, a series of dictionaries is used to label each team with the following information:
* prelim group placement
* prelim PPB

The reverse dictionary of the first one is also created, named placement_team.
^ is this actually needed?


In [160]:
%%skip $should_skip

team_placement = {rows[0]: (rows[1], str(rows[2])) for rows in list_of_teams}
team_ppb = {rows[0]: (rows[3]) for rows in list_of_teams}

# is this actually needed?
# placement_team = {(rows[1] + str(rows[2])): rows[0] for rows in list_of_teams}

In [161]:
%%skip $should_skip

advance_count = format_dict['the number of teams that advance to each bracket'
                            + ' (e.g. top 2 or top 3)']

crossover = format_dict['crossover']

# sort list by index 1 descending (finish), then index 2 ascending (PPB)
sorted_list = sorted(list_of_teams, key=lambda x: (x[2], -x[3], -x[4]))
if crossover != 'N':
    # ignore PPB if using crossover schedule
    # TODO I don't know if this is best method but it works for now
    sorted_list = sorted(list_of_teams, key=lambda x: (x[2]))

# # %% for visual debugging
# print('*** sorted_list ***\n')
# print(*sorted_list, sep='\n')
# if len(sorted_list) > 20:
#   print('continued...')

Reorganize sorted_list according to the playoff schedule codes 1 and 2:
* Code 1 is for reordering the list of teams according to the playoff format (most of the time, it will be in order from 1 through N).
* Code 2 is for splitting the list of teams into different brackets (most of the time, it will be the same number repeated, but occasionally it will be different numbers if not all brackets have the same size).

In [162]:
%%skip $should_skip

schedule_code_1 = format_dict['playoff schedule code 1']
sorted_list = code_1_scheduler(sorted_list, schedule_code_1)

schedule_code_2 = format_dict['playoff schedule code 2']
sorted_list = code_2_scheduler(sorted_list, schedule_code_2)

Some tournament format information needed.
* How many rounds of prelims were there (and what number is the first round of playoffs)?
* Is this a crossover format?

We'll also create a series of dictionaries pertaining to playoff team seeding.

In [163]:
%%skip $should_skip

prelim_round_count = format_dict['number of prelim rounds '
                                 + '(do not include tiebreakers)']

crossover = format_dict['crossover']
if crossover == 'N':
  crossover = False

# %% generate playoff seed / team dictionary

# key is a playoff bracket name and a seed (i.e. Berg6)
# value is the corresponding team (i.e. Great Valley A for Berg6)
# also created a dictionary where the two values are team codes (i.e. GVA)
playoff_team_dict = {}
playoff_teamcode_dict = {}
teamcode_playoff_dict = {}

for index, bracket_name in enumerate(playoff_bracket_names):

  # # visual debugging
  # print(f'\n\nbracket_name = {bracket_name}')

  for index2, team in enumerate(sorted_list[index]):

    key = bracket_name + str(index2+1)
    value = team[0]
    playoff_team_dict[key] = value

    # # visual debugging
    # print(f'team = {team}')
    # print(f'key = {key} \nvalue = {value}\n')

    value = team_code_dict[value]
    playoff_teamcode_dict[key] = value

    # creating a dictionary where k:v pairs are flipped
    teamcode_playoff_dict[value] = key

Finally, the creation of the full schedule grid.

In [164]:
%%skip $should_skip

# %% create full schedule grid + first playoff room dict

full_schedule_grid = []
first_playoff_rooms = {}

# perform this process for each prelim group, add to full_schedule_grid
for bracket_name in playoff_bracket_names:

  # this schedule grid will contain all match information split up by line
  schedule_grid = (standard_schedule(bracket_name,
                                     playoff_teamcode_dict,
                                     playoff_room_dict,
                                     crossover=crossover,
                                     roundstart=prelim_round_count+1))

  full_schedule_grid.append(schedule_grid)

  # produce roomlist + bye if needed
  roomlist = [k for k, v in list(playoff_room_dict.items())
              if k.startswith(bracket_name)]
  roomlist = [playoff_room_dict[i] for i in roomlist]

  if schedule_grid[0][-1] == 'BYE':
    roomlist.append('BYE')

  # populate first_playoff_rooms dict
  first_row = schedule_grid[1]
  for index, team in enumerate(first_row[1:]):
    room_index = int(index//2)
    first_room = roomlist[room_index]
    first_playoff_rooms[team] = first_room

# # prints output for visual debugging
# for index, playoff_bracket in enumerate(playoff_bracket_names):
#     print(f'\nPlayoff Bracket: {playoff_bracket}')
#     # visual_debugger(full_schedule_grid[index])


# Superplayoff Scheduler

# PDF Writer

The list of group names needs to be defined prior to writing the PDF files.

In [165]:
phase_capitalized = tournament_phase.capitalize()
if tournament_phase == 'prelims':
  groups = prelim_group_names
elif tournament_phase == 'playoffs':
  groups = playoff_bracket_names
elif tournament_phase == 'superplayoffs':
  pass

First to generate: full schedule for all teams, all rooms.

In [166]:
pdf = PDF(format="letter")  # default is A4, need to specify US letter

# title page
pdf.add_page()

doc_title = f"{phase_capitalized} - Complete Schedule"
pdf.title_page(format_dict['tournament name'],
               format_dict['tournament location'],
               format_dict['tournament date'],
               doc_title)

# full schedule page
pdf.add_page()

pdf.page_header(format_dict['tournament name'],
                format_dict['tournament date'])

for index, grid in enumerate(full_schedule_grid):
  
  # write group name above grid
  group = groups[index]
  pdf.set_text_color(0)
  pdf.set_font(style='I', size=16)
  pdf.cell(w=0, h=None, txt=f'Group - {group}',
            align="L", new_x="LEFT", new_y="NEXT")
  pdf.ln(5)

  # print grid
  header = grid[0]
  schedule = grid[1:]
  pdf.full_schedule(header, schedule)
  
  # line break spacer
  pdf.ln(10)

pdf.page_footer()

pdf.output(f"full_{tournament_phase}_schedule.pdf")
pdf_bytes = pdf.output()
full_schedule = b64encode(pdf_bytes).decode("utf-8")

Next, we'll generate the individualized team schedules.

In [167]:
pdf = PDF(format="letter")  # default is A4, need to specify US letter

# title page
pdf.add_page()

doc_title = f"{phase_capitalized} - Individual Team Schedules"
pdf.title_page(format_dict['tournament name'],
               format_dict['tournament location'],
               format_dict['tournament date'],
               doc_title)

# sort list of teams alphabetically, so documents easier to distribute
list_of_teams = sorted(list_of_teams, key=lambda x: (x[0]))

for team in list_of_teams:

  # iterate through schedule grids. Is the team in that grid?
  for index, schedule_grid in enumerate(full_schedule_grid):

    team_name = team[0]  # full team name 
    team_code = team_code_dict[team_name]  # 3-letter team code
    team_list = get_team_list(schedule_grid)  # list of teams in grid

    if team_code in team_list:
      group = groups[index]  # group name
      room_list = get_room_list(schedule_grid)
      basic_schedule_grid = clean_up_grid(schedule_grid)
      
      # create page and add header
      pdf.add_page()
      pdf.page_header(format_dict['tournament name'],
                      format_dict['tournament date'])
      
      # add team name, a line break, and group name
      pdf.set_font("times", size=24)
      pdf.cell(w=0, h=None, txt=f'{tournament_phase.capitalize()} Schedule: {team_name}',
                align="C", new_x="LEFT", new_y="NEXT")
      
      if single_prelim_group == True:   
        pdf.ln(15)
      if single_prelim_group == False:   
        pdf.ln(5)
        pdf.set_font("times", size=17)
        pdf.cell(w=0, h=None, txt=f'Group: {group}',
                  align="C", new_x="LEFT", new_y="NEXT")
        pdf.ln(10)

      grid = specific_team_scheduler(team_code,
                                     basic_schedule_grid,
                                     room_list,
                                     code_team_dict)

      header = grid[0]
      schedule = grid[1:]
      pdf.individual_schedule(header, schedule)

      if text_toggle is True:

        # todo: figure out latin-1 encoding issue
        text = texts[0]
        text = text.encode('latin-1', 'ignore').decode('latin-1')

        pdf.set_x(25.45)
        font_size = 14
        pdf.set_font("times", size=font_size)
        pdf.set_text_color(0)
        pdf.multi_cell(165, 6, text, new_x="LMARGIN")

      if qr_toggle is True:
          qr_codes_1 = qr_codes[0:3]
          qr_captions_1 = qr_captions[0:3]

          qr_positions = [35, 93, 151]
          for index, qr_code in enumerate(qr_codes_1):
            img = qrcode.make(qr_code)
            x_pos = qr_positions[index]
            pdf.image(img.get_image(), x=x_pos, y=210, w=30, h=30)

          for index, qr_caption in enumerate(qr_captions_1):
            x_pos = qr_positions[index] - 5
            pdf.set_xy(x_pos, 241)
            pdf.cell(w=40, h=None, txt=qr_caption,
                     align="C")

      pdf.page_footer()

      break  # saves time by ending search for team name in other schedule grids

pdf.output(f"{tournament_phase}_individual_team_schedules.pdf")
pdf_bytes = pdf.output()
individual_team_schedules = b64encode(pdf_bytes).decode("utf-8")

We'll do the same individual schedules, but for rooms instead of teams.

In [168]:
pdf = PDF(format="letter")  # default is A4, need to specify US letter

# title page
pdf.add_page()

doc_title = f"{phase_capitalized} - Individual Room Schedules"
pdf.title_page(format_dict['tournament name'],
               format_dict['tournament location'],
               format_dict['tournament date'],
               doc_title)

for index, schedule_grid in enumerate(full_schedule_grid):

  group = groups[index]
  room_list = get_room_list(schedule_grid)
  basic_schedule_grid = clean_up_grid(schedule_grid)

  # iterate for each room in room_list
  for room in room_list:
    if room == 'BYE':
      continue
  
    # create page and add header
    pdf.add_page()
    pdf.page_header(format_dict['tournament name'],
                    format_dict['tournament date'])

    # add team name, a line break, and prelim group name
    pdf.set_font("times", size=24)
    
    pdf.cell(w=0, h=None, txt=f'{tournament_phase.capitalize()} Schedule: {room}',
              align="C", new_x="LEFT", new_y="NEXT")

    # skip the prelim group name if only one prelim group
    if single_prelim_group == True:   
      pdf.ln(15)
    if single_prelim_group == False:   
      pdf.ln(5)
      pdf.set_font("times", size=17)
      pdf.cell(w=0, h=None, txt=f'Group: {group}',
                align="C", new_x="LEFT", new_y="NEXT")
      pdf.ln(10)

    grid = specific_room_scheduler(room,
                                   basic_schedule_grid,
                                   room_list,
                                   code_team_dict)

    header = grid[0]
    schedule = grid[1:]
    pdf.individual_schedule(header, schedule)

    if text_toggle is True:

        # todo: figure out latin-1 encoding issue
        text = texts[1]
        text = text.encode('latin-1', 'ignore').decode('latin-1')

        pdf.set_x(25.45)
        font_size = 14
        pdf.set_font("times", size=font_size)
        pdf.set_text_color(0)
        pdf.multi_cell(165, 6, text, new_x="LMARGIN")

    if qr_toggle is True:
        qr_codes_2 = qr_codes[3:6]
        qr_captions_2 = qr_captions[3:6]

        qr_positions = [35, 93, 151]
        for index, qr_code in enumerate(qr_codes_2):
          img = qrcode.make(qr_code)
          x_pos = qr_positions[index]
          pdf.image(img.get_image(), x=x_pos, y=210, w=30, h=30)

        for index, qr_caption in enumerate(qr_captions_2):
          x_pos = qr_positions[index] - 5
          pdf.set_xy(x_pos, 241)
          pdf.cell(w=40, h=None, txt=qr_caption,
                    align="C")

    pdf.page_footer()

pdf.output(f"{tournament_phase}_individual_room_schedules.pdf")
pdf_bytes = pdf.output()
individual_room_schedules = b64encode(pdf_bytes).decode("utf-8")

In [169]:
class PDF(PDF):

    def team_index(self, list_of_teams, phase='prelims',
                   header_color=(135, 135, 135), row_color=(210, 210, 210)):

      # depending on phase, create header and cell widths
      if phase == 'prelims':
        header = ['Team Name', 'Team Code', 'Prelim Group']
        cell_widths = [66, 33, 66]

      elif phase == 'playoffs':
        header = ['Team Name', 'Prelim Group', 'Finish',
                  'PPB', 'Playoff Bracket']
        cell_widths = [47, 23.5, 23.5, 23.5, 47]
      elif phase == 'superplayoffs':
        header = ['Team Name', 'Playoff Bracket', 'Finish',
                  'PPB*', 'Super-playoff Bracket', 'W-L']  # may be too long
        cell_widths = [30, 45, 15, 15, 45, 15]
        

      # start table at 25.45, draw cells depending on cell widths
      # right margin will also be 25.45

      self.set_x(25.45)

      self.set_fill_color(*header_color)  # * required to unpack tuple
      self.set_text_color(255)
      self.set_draw_color(36, 36, 36)
      self.set_line_width(0.3)
      self.set_font(style="B", size=12)
      for cell_width, heading in zip(cell_widths, header):
          self.cell(cell_width, 8, heading, border=1, align="C", fill=True)
      self.ln()
      self.set_x(25.45)

      # Color and font restoration:
      self.set_fill_color(*row_color)  # * required to unpack tuple
      self.set_text_color(0)
      self.set_font()
      fill = False
      if len(list_of_teams) >= 12:
        font_size = 10
        row_height = 6
      else:
        font_size = 12
        row_height = 8
      for team in list_of_teams:
        print(team)
        continue
        for index, item in enumerate(team):       
          if index == 0:
            self.set_font(style="B", size=font_size)
          else:
            self.set_font(size=font_size)
          self.cell(cell_widths[index],
                    row_height,
                    item,
                    border="LR",
                    align="C",
                    fill=fill)
        fill = not fill  # I am pretty sure this is what alternates row colors!

# *** Latex Writer Code ***

#     for i in list_of_teams:
#         teamcode = team_code_dict[i[0]]
#         playoff_seed = teamcode_playoff_dict[teamcode][-1]
#         prelim_finish = i[2]
#         ppb = '%s' % '%.2f' % i[3]
#         playoff_bracket = teamcode_playoff_dict[teamcode][:-1]
#         team = playoff_team(i[0], teamcode, i[1],
#                             playoff_bracket, playoff_seed)
#         table.add_row(team.name,
#                       team.prelim_group, prelim_finish,
#                       ppb, team.playoff_bracket)
        self.ln()
        self.set_x(25.45)

      self.set_fill_color(*header_color)  # * required to unpack tuple
      self.set_text_color(255)
      self.set_draw_color(36, 36, 36)
      self.set_line_width(0.3)
      self.set_font(style="B", size=12)
      for cell_width, heading in zip(cell_widths, header):
          self.cell(cell_width, 8, heading, border=1, align="C", fill=True)

      self.ln(14)

In [170]:
# team index class, in progress

pdf = PDF(format="letter")  # default is A4, need to specify US letter

# title page
pdf.add_page()

doc_title = f"{phase_capitalized} - Team Index"
pdf.title_page(format_dict['tournament name'],
               format_dict['tournament location'],
               format_dict['tournament date'],
               doc_title)

# full schedule page
pdf.add_page()

pdf.page_header(format_dict['tournament name'],
                format_dict['tournament date'])

pdf.team_index(list_of_teams, phase=tournament_phase)

pdf.page_footer()

pdf.output(f"{tournament_phase}_team_index.pdf")
pdf_bytes = pdf.output()
team_index = b64encode(pdf_bytes).decode("utf-8")

['Amherst', 'A', 5, 0, 14.35]
['Bard College', 'B', 6, 0, 13.5]
['Bates', 'A', 6, 0, 11.62]
['NYU A', 'A', 2, 0, 20.44]
['NYU B', 'B', 2, 0, 19.49]
['Penn', 'B', 5, 0, 14.19]
['Princeton A', 'B', 4, 0, 14.9]
['Princeton B', 'A', 4, 0, 17.33]
['Rutgers A', 'A', 3, 0, 20.07]
['Rutgers B', 'B', 3, 0, 18.81]
['Yale A', 'B', 1, 0, 21.14]
['Yale B', 'A', 1, 0, 23.59]


## Download PDFs

In [171]:
# Diplay a download button:
display(HTML(f'<a style="font-size: 30px" download="{tournament_phase}_full_schedule.pdf" href="data:application/pdf;base64,{full_schedule}">Click to download PDF of full schedule.</a>'))

In [172]:
# Diplay a download button:
display(HTML(f'<a style="font-size: 30px" download="{tournament_phase}_individual_team_schedules.pdf" href="data:application/pdf;base64,{individual_team_schedules}">Click to download PDF of individual team schedules.</a>'))

In [173]:
# Diplay a download button:
display(HTML(f'<a style="font-size: 30px" download="{tournament_phase}_individual_room_schedules.pdf" href="data:application/pdf;base64,{individual_room_schedules}">Click to download PDF of individual room schedules.</a>'))

In [174]:
stop here

SyntaxError: ignored

# NSC Scheduler

# *** Workspace below this point ***



---



# Importing csv files from Github
This will be useful later when grabbing templates for the scheduling functions.

For more info on loading external files into Colabs, see [this article](https://towardsdatascience.com/7-ways-to-load-external-data-into-google-colab-7ba73e7d5fc7).

In [ ]:
filename = 'templates/rr_schedules/rr_10.xlsx'

df = pd.read_csv('https://raw.githubusercontent.com/victorprieto2265/cornerstone/main/archive/brackets.csv')

# get_all_values gives a list of rows.
print(df)

What about uploading excel files? Will that work?

In [ ]:
url = "https://github.com/victorprieto2265/cornerstone/blob/main/templates/rr_schedules/rr_templates.xlsx?raw=true"
db=pd.read_excel(url, sheet_name=None)

print(type(db))
print(db.keys())